In [ ]:
!pip install --upgrade gmplot

In [1]:
from pyspark.conf import SparkConf
import gmplot

from src.GPSProcessing import *

In [ ]:
# REFERENCE: https://spark.apache.org/docs/latest/configuration.html

conf = SparkConf().setAll([('spark.memory.fraction','0.6'),
                           ('spark.executor.memory', '16g'),
                           ('spark.driver.memory','16g'),
                           ('spark.sql.shuffle.partitions','20'),
                           ('spark.memory.offHeap.enabled', True),
                           ('spark.memory.offHeap.size','16g'),
                           ('spark.cleaner.referenceTracking.cleanCheckpoints', True),
                           ('spark.driver.host','127.0.0.1')]
                         )

spark  = SparkSession.builder.config(conf=conf).master("local[*]").appName("trip trajectories").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
sc.setCheckpointDir('checkpoints')
sc.getConf().getAll()

In [ ]:
sc.defaultParallelism

In [ ]:
sc.defaultMinPartitions

In [ ]:
sc

## Load GPS processed data

In [6]:
gps_path = '../test_data/merged/habitus_gps_acc_1.csv'

In [7]:
gps_data = spark.read.csv(gps_path, header=True, inferSchema=True)

In [ ]:
gps_data.show()

In [9]:
max_trip_value = gps_data.agg({"tripNumber": "max"}).collect()[0][0]

In [ ]:
max_trip_value

In [11]:
dataframe = gps_data.select(['lat','lon']).toPandas()

lat_list = dataframe['lat'].tolist()
lon_list = dataframe['lon'].tolist()

In [12]:
min_lat, max_lat, min_lon, max_lon = min(lat_list), max(lat_list), min(lon_list), max(lon_list)

gmap = gmplot.GoogleMapPlotter(min_lat + (max_lat - min_lat) / 2, 
                               min_lon + (max_lon - min_lon) / 2, 
                               16)

In [13]:
for n in range(100):

    trip_set = gps_data.filter(F.col('tripNumber') == n + 1).select(['lat','lon'])

    lat_list = trip_set.toPandas()['lat'].tolist()
    lon_list = trip_set.toPandas()['lon'].tolist()
    
    gmap.scatter(lat_list, lon_list, 'red', size = 0.5, marker = False)
    gmap.plot(lat_list, lon_list, 'blue', edge_width = 3) 

In [14]:
gmap.draw('gmap.html')

In [15]:
# import gmplot package
import gmplot

# GoogleMapPlotter return Map object
# Pass the center latitude and
# center longitude
gmap1 = gmplot.GoogleMapPlotter(30.3164945,
                                78.03219179999999, 13 )

# Pass the absolute path
gmap1.draw("map11.html")